In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.insert(0,"../../python/")
from parametrization import ParamHelper

In [ ]:
sys.path.append('../../python/')
import centrality_utils.weight_funtions as wf 
import centrality_utils.online_rank_computer as olrc
import centrality_utils.online_indegree_computer as olidc
import centrality_utils.temporal_pagerank as tprc
import centrality_utils.static_pagerank_computer as sprc
import centrality_utils.static_indegree_computer as sidc
import centrality_utils.static_negative_beta_measure_computer as snbmc
import centrality_utils.static_harmonic_centrality_computer as shcc
from centrality_utils.online_rank_computer import link2str

import simulator_utils.edge_simulator as esim
import simulator_utils.graph_simulator as gsim

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load Parameters

In [ ]:
ph = ParamHelper("../../pipelines/OnlineCentrality.json", "ipython/experiments/centrality_score_computer.ipynb")

In [ ]:
file_prefix = ph.get("data_file_prefix")
score_output_prefix = ph.get("score_folder_prefix")
dataset_id = ph.get("dataset_id")
time_type = ph.get("time_type")
experiment_path = ph.get("experiment_path")
min_epoch = ph.get("min_epoch")

In [ ]:
print(experiment_path)

In [ ]:
if time_type == "index":
    index_threshold = ph.get("edge_index_threshold")
    delta = ph.get("edge_delta")
else:
    index_threshold = ph.get("day_index_threshold")
    delta = ph.get("epoch_delta")
num_of_intervals_plus_one = ph.get("day_index_threshold")
print(delta,index_threshold)

# 2. Load Graph Data 

In [ ]:
data_path = file_prefix + '/%s_mentions.csv' % dataset_id
data = np.loadtxt(data_path, delimiter=' ', dtype='i')
print('%s dataset were loaded.' % dataset_id)
print('Number of edges in data: %i.' % len(data))

In [ ]:
data[:5]

## a.) exclude early information

In [ ]:
import numpy as np
selector = data[:,0] >= min_epoch
data = data[selector,:]
print('Number of edges in data after excluding edges below epoch %i: %i.' % (min_epoch,len(data)))

## b.) preprocessing nodes and edges

In [ ]:
src_unique = np.unique(data[:,1])
trg_unique = np.unique(data[:,2])
nodes = np.unique(np.concatenate((src_unique,trg_unique)))
edges = [link2str(link) for link in data[:,1:3].tolist()] # element must be string to be hashable
len(nodes), len(edges)

In [ ]:
edges[:3]

# 3. Compute online centraliy measures

## a.) Setting parameters

In [ ]:
olr_params, tpr_params, pr_params, indeg_params, nbm_params, hc_params = [], [], [], [], [], []
gsim_params = []

norm_factors = [1.0]
norm_factors += [3600.0 * i for i in [0.25,0.5,0.75,1,1.25,1.5,1.75,2,3,4,5,6,7,8,9,10,11,12,18,24]]
norm_factors += [86400.0 * i for i in [2,3,4,5,6,7,14,21,30]]
print(norm_factors)

In [ ]:
norm_factors = [1.0]
norm_factors += [3600.0 * i for i in [1,2,3,4,5,6,7,8,9,10,11,12,24]]
print(norm_factors)

In [ ]:
if delta == 3600:
    static_lookbacks = list(range(1,25))
else:
    static_lookbacks = [0, 1, 2, 3, 4, 5, 6, 7, 14, 21, 30]

### Select parameters for OnlineRankComputer

In [ ]:
olr_beta = ph.get("olr_beta")
print(olr_beta)

In [ ]:
olr_params = []
olr_params += [olrc.OnlineRankParams(0.05,olr_beta,wf.ConstantWeighter(1.0))]
olr_params += [olrc.OnlineRankParams(0.05,olr_beta,wf.ExponentialWeighter(base=0.5,norm=n)) for n in norm_factors]

In [ ]:
if len(olr_params) > 0:
    if time_type == "index":
        gsim_params.append(olrc.OnlineRankComputer(nodes,edges,olr_params))
    else:
        gsim_params.append(olrc.OnlineRankComputer(nodes,edges,olr_params,min_time=min_epoch))

### Select parameters for TemporalPageRankComputer

In [ ]:
tpr_params += [tprc.TemporalPageRankParams(0.85,b) for b in [0.001,0.01,0.05,0.1,0.3,0.5,0.9]] 

In [ ]:
if len(tpr_params) > 0:
    gsim_params.append(tprc.TemporalPageRankComputer(nodes,tpr_params))

### Select parameters for StaticPageRankComputer

In [ ]:
pr_params += [sprc.StaticPageRankParams(lookback_cnt=l,alpha=0.85,max_iter=100) for l in static_lookbacks]

In [ ]:
if len(pr_params) > 0:
    gsim_params.append(sprc.StaticPageRankComputer(pr_params))

### Select parameters for StaticIndegreeComputer

In [ ]:
indeg_params += [sidc.StaticIndegreeParams(lookback_cnt=l) for l in static_lookbacks]

In [ ]:
if len(indeg_params) > 0:
    gsim_params.append(sidc.StaticIndegreeComputer(indeg_params))

### Select parameters for StaticNegativeBetaMeasureComputer

In [ ]:
nbm_params += [snbmc.StaticNegativeBetaMeasureParams(lookback_cnt=l) for l in static_lookbacks]

In [ ]:
if len(nbm_params) > 0:
    gsim_params.append(snbmc.StaticNegativeBetaMeasureComputer(nbm_params))

### Select parameters for StaticHarmonicCentralityComputer

In [ ]:
#exclude computation on the total graph
for l in static_lookbacks:
    if l == 0:
        continue
    else:
        hc_params.append(shcc.StaticHarmonicCentralityParams(lookback_cnt=l))

In [ ]:
if len(hc_params) > 0:
    gsim_params.append(shcc.StaticHarmonicCentralityComputer(hc_params))

### Select parameters for OnlineIndegreeComputer

In [ ]:
batch_prefix = ph.get("batch_prefix")
print(batch_prefix)

In [ ]:
olid_params = []
olid_params += [olidc.OnlineIndegreeParams(wf.ExponentialWeighter(base=0.5,norm=n), batch_score_part=batch_prefix) for n in norm_factors]

In [ ]:
if len(olid_params) > 0:
    if time_type == "index":
        gsim_params.append(olidc.OnlineIndegreeComputer(nodes,edges,olid_params))
    else:
        gsim_params.append(olidc.OnlineIndegreeComputer(nodes,edges,olid_params,min_time=min_epoch))

## b.) Compute all online scores with one graph simulation

In [ ]:
boundaries = min_epoch + np.array([delta*i for i in range(1,num_of_intervals_plus_one)])
len(boundaries)

In [ ]:
%%time
gsim_obj = gsim.OnlineGraphSimulator(data, time_type=time_type, verbose=True)
experiment_graph_stats = gsim_obj.run_with_boundaries(gsim_params,boundaries,experiment_path,max_index=index_threshold)